<a href="https://colab.research.google.com/github/Ele975/breast_cancer_detection/blob/main/Breast_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [1]:
!pip install kaggle

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

Download first locally the .json file from https://www.kaggle.com/datasets/yasserh/breast-cancer-dataset

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elenafranchini","key":"f3c528078b7998afd62ab73c0905f6a3"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# Download the dataset
!kaggle datasets download -d uciml/breast-cancer-wisconsin-data

# Unzip the dataset
!unzip breast-cancer-wisconsin-data.zip

Dataset URL: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
License(s): CC-BY-NC-SA-4.0
  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 1.17MB/s]
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


## Data analysis and cleaning

Read dataset

In [6]:
df = pd.read_csv("data.csv")
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

Remove NaN column "Unnamed: 32"

In [8]:
df = df.drop("Unnamed: 32", axis=1)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Check for other NaN

In [9]:
print(df.isnull().any())

id                         False
diagnosis                  False
radius_mean                False
texture_mean               False
perimeter_mean             False
area_mean                  False
smoothness_mean            False
compactness_mean           False
concavity_mean             False
concave points_mean        False
symmetry_mean              False
fractal_dimension_mean     False
radius_se                  False
texture_se                 False
perimeter_se               False
area_se                    False
smoothness_se              False
compactness_se             False
concavity_se               False
concave points_se          False
symmetry_se                False
fractal_dimension_se       False
radius_worst               False
texture_worst              False
perimeter_worst            False
area_worst                 False
smoothness_worst           False
compactness_worst          False
concavity_worst            False
concave points_worst       False
symmetry_w

Convert diagnosis (target) to binary labels

In [10]:
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Apply standardization (better than normalization in this case)

In [11]:
def standardization(df):
  for column in df.columns:
    if column != "id" and column != "diagnosis":
      df[column] = (df[column] - df[column].mean()) / df[column].std()

In [12]:
standardization(df)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,...,1.885031,-1.358098,2.301575,1.999478,1.306537,2.614365,2.107672,2.294058,2.748204,1.935312
1,842517,1,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,...,1.804340,-0.368879,1.533776,1.888827,-0.375282,-0.430066,-0.146620,1.086129,-0.243675,0.280943
2,84300903,1,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,...,1.510541,-0.023953,1.346291,1.455004,0.526944,1.081980,0.854222,1.953282,1.151242,0.201214
3,84348301,1,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,...,-0.281217,0.133866,-0.249720,-0.549538,3.391291,3.889975,1.987839,2.173873,6.040726,4.930672
4,84358402,1,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,...,1.297434,-1.465481,1.337363,1.219651,0.220362,-0.313119,0.612640,0.728618,-0.867590,-0.396751


Check class balance

In [13]:
print(df["diagnosis"].value_counts())

diagnosis
0    357
1    212
Name: count, dtype: int64


class imbalance ratio -> the classes are slightly imbalanced. We can use the parameter 'class_weight = "balanced"' in the sklearn models (weight each class example proportionally to the inverse of its frequency), however by empirical experiments the score decreased, thus no balancing is performed.

In [14]:
imbalance = df["diagnosis"].value_counts()[0] / df["diagnosis"].value_counts()[1]
print(f"Imbalance ratio: {imbalance}")

Imbalance ratio: 1.6839622641509433


In [15]:
random_state = 42

Split features and targets

In [16]:
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

In [17]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=random_state)
print(train_x.shape[0])
print(test_x.shape[0])

455
114


## Models comparison

Grid search method generalized for all models

In [18]:
def grid_search(model, param_grid, X, y):
  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
  grid_search.fit(X, y)
  best_params = grid_search.best_params_
  best_score = grid_search.best_score_
  print(f"Best training score:\n{grid_search.best_score_}")
  print(f"Best parameters: {best_params}")
  return grid_search.best_estimator_, best_params, best_score

Dictionary with {model: [(best_params1, best_score1),(best_params1, best_score1), ...]}

In [19]:
best_res = {}

### Decision tree
Use a decision tree since the dataset is small.
Train the model usign the training set

In [20]:
decision_tree = DecisionTreeClassifier(random_state=random_state)

**Hyperparameter tuning**

Hyperparameters of decision trees:
1. depth of the tree
2. minimum number of samples required to split a node
3. minimum number of samples for terminal nodes
3. criterion: function to measure the quality of a split

In [21]:
param_grid_dt = {
    'max_depth': [None, 2, 5, 10, 20],
    'min_samples_split' : [2, 5, 10, 20, 30],
    'min_samples_leaf' : [1,2,4],
    'criterion': ['gini', 'entropy']
}

In [22]:
opt_dc = grid_search(decision_tree, param_grid_dt, train_x, train_y)
pred_y = opt_dc[0].predict(test_x)
# accuracy = accuracy_score(test_y, pred_y)
# print(f"Accuracy:\n{accuracy}")

Best training score:
0.9472527472527472
Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [23]:
# update dictionary of global results
best_res[opt_dc[0]] = [(opt_dc[1], opt_dc[2])]

### Logistic regression

In [24]:
logreg = LogisticRegression(random_state=random_state)

For the logistic regression models, some hyperparameters cannot be combined with others, thus we separate all combinations based on the penalty parameter

In [25]:
param_grid_logreg = [
    {
        # l1 penalty only works with 'liblinear' and 'saga'
        'penalty': ['l1'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['liblinear', 'saga'],
        'max_iter': [100, 1000, 2500, 5000],
    },
    {
        # l2 penalty works with most solvers
        'penalty': ['l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': [100, 1000, 2500, 5000],
    },
    {
        # elasticnet requires 'saga' solver and l1_ratio
        'penalty': ['elasticnet'],
        # Mandatory for elasticnet
        'l1_ratio': [0.1, 0.5, 0.9],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['saga'],
        'max_iter': [100, 1000, 2500, 5000],
    },
    {
        # penalty=None is only supported by 'saga'
        'penalty': [None],
        'solver': ['saga'],
        'max_iter': [100, 1000, 2500, 5000],
    }
]


In [26]:
for params in param_grid_logreg:
  opt_logreg = grid_search(logreg, params, train_x, train_y)
  pred_y = opt_logreg[0].predict(test_x)

  if opt_logreg[0] in best_res:
    best_res[opt_logreg[0]].append((opt_logreg[1], opt_logreg[2]))
  else:
    best_res[opt_logreg[0]] = [(opt_logreg[1], opt_logreg[2])]

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Best training score:
0.9758241758241759
Best parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: Rounding errors prevent the line search from converging
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/optimize.py:312: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/optimize.p

Best training score:
0.9758241758241759
Best parameters: {'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}
Best training score:
0.5274725274725275
Best parameters: {'C': 0.001, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'elasticnet', 'solver': 'saga'}
Best training score:
0.5274725274725275
Best parameters: {'max_iter': 100, 'penalty': None, 'solver': 'saga'}


### Random forest
The random forest combines multiple decision trees, where each of them receives a random split of the training and test data. At the end, the result reached by the majority of decision trees will be selected as the final one.

In [27]:
rnd_forest = RandomForestClassifier(class_weight = 'balanced', random_state=random_state)

In [28]:
param_grid_rnd_forest = {
    'n_estimators': [10, 50, 100, 200, 300],
    'max_depth': [None, 10, 20, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [29]:
opt_rnd_forest = grid_search(rnd_forest, param_grid_rnd_forest, train_x, train_y)
pred_y = opt_rnd_forest[0].predict(test_x)
# accuracy = accuracy_score(test_y, pred_y)
# print(f"Accuracy:\n{accuracy}")

Best training score:
0.9582417582417582
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}


In [30]:
# update dictionary of global results
best_res[opt_rnd_forest[0]] = [(opt_rnd_forest[1], opt_rnd_forest[2])]

Get all results

In [31]:
for key, value in best_res.items():
  print(key, value)

DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42) [({'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}, 0.9472527472527472)]
LogisticRegression(C=1, penalty='l1', random_state=42, solver='liblinear') [({'C': 1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}, 0.9758241758241759)]
LogisticRegression(C=10, max_iter=1000, random_state=42, solver='newton-cg') [({'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}, 0.9758241758241759)]
LogisticRegression(C=0.001, l1_ratio=0.1, penalty='elasticnet', random_state=42,
                   solver='saga') [({'C': 0.001, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'elasticnet', 'solver': 'saga'}, 0.5274725274725275)]
LogisticRegression(penalty=None, random_state=42, solver='saga') [({'max_iter': 100, 'penalty': None, 'solver': 'saga'}, 0.5274725274725275)]
RandomForestClassifier(class_weight='balanced', min_samples_split=10,
                       n_estimators=50,

Best model: Logistic Regression with C=1, penalty='l1', solver='liblinear', max_iter=100